In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import jieba.analyse
from bert_serving.client import BertClient

/Users/yitinglin/Projects/news-emotion-analysis/env/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
bert_client = BertClient()

In [3]:
content_df = pd.read_csv('../data/Train_Dataset.csv')
label_df = pd.read_csv('../data/Train_Dataset_Label.csv')

In [4]:
train_df = content_df.merge(label_df, on='id')
train_df = train_df.fillna('EMPTY')
train_df.shape

(7340, 4)

In [7]:
str_list = []
for i in range(train_df.shape[0]):
    title = train_df.title.values[i]
    content = train_df.content.values[i]
    content_ad_keyword = jieba.analyse.textrank(title+content, topK=50, withWeight=False, allowPOS=('a'))
    content_nv_keyword = jieba.analyse.textrank(title+content, topK=50, withWeight=False, allowPOS=('n', 'v'))
    keyword_str = ' '.join(content_ad_keyword + content_nv_keyword)
    if len(keyword_str) == 0:
        str_list.append('EMPTY')
    else:
        str_list.append(keyword_str)

In [10]:
train_data = bert_client.encode(str_list)
print(train_data.shape)
pd.DataFrame(train_data).to_csv('./train_keyword_word_vector.csv', header=None, index=None)

(7340, 768)


In [11]:
test_df = pd.read_csv('../data/Test_Dataset.csv')
test_df = test_df.fillna('EMPTY')
test_df.shape

(7356, 3)

In [12]:
str_list = []
for i in range(test_df.shape[0]):
    title = test_df.title.values[i]
    content = test_df.content.values[i]
    content_ad_keyword = jieba.analyse.textrank(title+content, topK=50, withWeight=False, allowPOS=('a'))
    content_nv_keyword = jieba.analyse.textrank(title+content, topK=50, withWeight=False, allowPOS=('n', 'v'))
    keyword_str = ' '.join(content_ad_keyword + content_nv_keyword)
    if len(keyword_str) == 0:
        str_list.append('EMPTY')
    else:
        str_list.append(keyword_str)

In [13]:
test_data = bert_client.encode(str_list)
print(test_data.shape)
pd.DataFrame(test_data).to_csv('./test_keyword_word_vector.csv', header=None, index=None)

(7356, 768)
